<a href="https://colab.research.google.com/github/prakashaditya369/AdditionQuiz/blob/master/Beautiful_II.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install mtcnn
!pip install ray[tune]

In [ ]:
#@title Default title text
!unzip /content/drive/MyDrive/Beauty/dataset.zip

# Preprocessing

In [ ]:
#@title Helper Functions
def progress_batch(i,Len, width=30):
  left = int(width * (i+1)*100/Len) // 100
  right = width - left
  print('\r[', '*' * left, ' ' * right, ']',f'{str(i+1)}/{str(Len)}',
        sep='', end='', flush=True)
def progress(percent=0, width=30):
  left = int(width * percent) // 100
  right = width - left
  print('\r[', '#' * left, ' ' * right, ']',
        f' {percent:.2f}%',
        sep='', end='', flush=True)

In [ ]:
import cv2
import os
from functools import partial
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler
# from mtcnn import MTCNN
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
# CUDA_LAUNCH_BLOCKING=1
# detector = MTCNN()

In [ ]:
def preprocess_image(img_add):
  img = cv2.cvtColor(cv2.imread(img_add), cv2.COLOR_BGR2RGB)
  result = detector.detect_faces(img)
  if len(result)>0:
    result = result[0]
    keypoints = result['keypoints']
    leftEyeCenter = keypoints['left_eye']
    rightEyeCenter = keypoints['right_eye']
    dY = rightEyeCenter[1] - leftEyeCenter[1]
    dX = rightEyeCenter[0] - leftEyeCenter[0]
    angle = np.degrees(np.arctan2(dY, dX))
    desiredFaceWidth = 227
    desiredFaceHeight = 227
    desiredLeftEye=(0.33, 0.35)
    desiredRightEyeX = 1.0 - desiredLeftEye[0]
    dist = np.sqrt((dX ** 2) + (dY ** 2))
    desiredDist = (desiredRightEyeX - desiredLeftEye[0])
    desiredDist *= desiredFaceWidth
    scale = desiredDist / dist
    eyesCenter = ((leftEyeCenter[0] + rightEyeCenter[0]) // 2, (leftEyeCenter[1] + rightEyeCenter[1]) // 2)
    M = cv2.getRotationMatrix2D(eyesCenter, angle, scale)
    tX = desiredFaceWidth * 0.5
    tY = desiredFaceHeight * desiredLeftEye[1]
    M[0, 2] += (tX - eyesCenter[0])
    M[1, 2] += (tY - eyesCenter[1])
    (w, h) = (desiredFaceWidth, desiredFaceHeight)
    (y2,x2,y1,x1) = result['box']
    img = cv2.warpAffine(img, M, (w, h),flags=cv2.INTER_CUBIC)
  return img

In [ ]:
def preprocess():
  print("Reading Files")
  CSV_FILE = "/content/dataset/dataset.csv"
  IMG_PATH = "/content/dataset/images/"
  final_image_size = (227, 227)
  df = pd.read_csv(CSV_FILE)
  labels = list(df["Score"])
  labels = [i-1 for i in labels]
  image_add = df["Image"]
  images = []
  i = 0
  print("Face Detecting and building the array.")
  for image in image_add:
    progress((i+1)/len(image_add)*100)
    img = preprocess_image(IMG_PATH+image)
    images.append(img)
    i+=1
  return image_add, images, labels

In [ ]:
add, images, labels = preprocess()

In [ ]:
images = np.load("/content/drive/MyDrive/Beauty/beauty_images.npy")
labels = np.load("/content/drive/MyDrive/Beauty/beauty_labels.npy")
# greyed_images = np.dot(images[:,:,:,:3], [0.299, 0.587, 0.114]).reshape(images.shape[0],227,227,1)

In [ ]:
#Data Augmentation part
greyed_imaged = np.dot(images[:,:,:,:3], [0.299, 0.587, 0.114])
print(greyed_imaged.shape)

In [ ]:
temp_grey_images = []
for i in range(greyed_imaged.shape[0]):
  imgh = greyed_imaged[i]
  imgs = np.flip(imgh, 1)
  temp_grey_images.append(imgs)
temp_grey_images = np.array(temp_grey_images)
greyed_images = np.concatenate((greyed_imaged, temp_grey_images)).reshape(2*images.shape[0], 128, 128, 1)
labels = np.concatenate((labels, labels))
np.save("/content/drive/MyDrive/Beauty/grey_augment_images.npy", greyed_images)
np.save("/content/drive/MyDrive/Beauty/augment_labels.npy", labels)

In [ ]:
greyed_images = np.load("/content/drive/MyDrive/Beauty/grey_augment_images.npy")
labels = np.load("/content/drive/MyDrive/Beauty/augment_labels.npy")

In [ ]:
shuffle = np.random.randint(0,greyed_images.shape[0], 20)
for i in shuffle:
  plt.imshow(greyed_images[i][:,:,0])
  plt.show()
  print(labels[i])

In [ ]:
shit = []
label = []
for i in range(len(images)):
  if images[i] is not None:
    shit.append(cv2.resize(images[i], (227,227), interpolation = cv2.INTER_AREA))
    label.append(labels[i])
labels = np.array(label)
images = np.array(shit)
np.save("/content/drive/MyDrive/Beauty/beauty_images", images)
np.save("/content/drive/MyDrive/Beauty/beauty_labels", labels)

In [ ]:
print(len(images),len(labels))

In [ ]:
class ImageLabel(Dataset):
  def __init__(self,image_tensor,label_tensor):
    self.image = image_tensor
    self.labels = label_tensor
  def __len__(self):
    return len(self.labels)
  def __getitem__(self, idx):
    return (self.image[idx],self.labels[idx])

In [ ]:
def get_dataset(images_cropped, labels, size = (256, 256)):
  #Distributing Data And Converting to Torch Tensor
  shuffle = np.random.permutation(len(labels))
  '''
  images = []
  labels = [i-1 for i in labels]
  for img in images_cropped:
    images.append(cv2.resize(img, size, interpolation = cv2.INTER_AREA))
  '''
  ratio = int(4/5*len(labels))
  train_num = shuffle[:ratio]
  val_num = shuffle[ratio:]
  print("Total Data: ", len(labels))
  '''
  train_label = torch.tensor(np.array(labels)[train_num])
  train_image = torch.tensor(np.array(images)[train_num].transpose(0,3,1,2), dtype=torch.float)
  val_label = torch.tensor(np.array(labels)[val_num])
  val_image = torch.tensor(np.array(images)[val_num].transpose(0,3,1,2), dtype = torch.float)
  '''
  train_label = torch.tensor(labels[train_num])
  train_image = torch.tensor(images_cropped[train_num].transpose(0,3,1,2), dtype=torch.float)
  val_label = torch.tensor(labels[val_num])
  val_image = torch.tensor(images_cropped[val_num].transpose(0,3,1,2), dtype = torch.float)
  # print("Training Data: ",train_label.shape, train_image.shape)
  # print("Validate Data: ", val_label.shape, val_image.shape)
  train_data = ImageLabel(train_image, train_label)
  val_data = ImageLabel(val_image, val_label)
  return train_data, val_data

# Model Section

## AlexNet

In [ ]:
class AlexNet(nn.Module):
    def __init__(self, l1 = 4096, l2 = 200):
        '''Input Image Size: (227, 227)'''
        super(AlexNet, self).__init__()
        self.conv1 = nn.Conv2d(in_channels = 1, out_channels = 96, kernel_size = 11)
        self.conv2 = nn.Conv2d(in_channels = 96, out_channels = 256, kernel_size = 5)
        self.conv3 = nn.Conv2d(in_channels = 256, out_channels = 384, kernel_size = 3)
        self.conv4 = nn.Conv2d(in_channels = 384, out_channels = 384, kernel_size = 3)
        self.conv5 = nn.Conv2d(in_channels = 384, out_channels = 256, kernel_size = 3)
        self.fc1 = nn.Linear(4096, l1)
        self.fc2 = nn.Linear(l1, l2)
        self.fc3 = nn.Linear(l2, 5)
    def forward(self, x, verbose=False):
        x = self.conv1(x)
        x = F.relu(x)
        x = F.max_pool2d(x, kernel_size=2)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, kernel_size=2)
        x = self.conv3(x)
        x = F.relu(x)
        x = F.max_pool2d(x, kernel_size=2)
        x = self.conv4(x)
        x = F.relu(x)
        x = F.max_pool2d(x, kernel_size=2)
        x = self.conv5(x)
        x = F.relu(x)
        x = F.max_pool2d(x, kernel_size=2)
        x = torch.flatten(x,1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)
        x = F.log_softmax(x, dim=1)
        return x

## Some Other Model

In [ ]:
class Model_FER(nn.Module):
  def __init__(self, l1= 128, l2 = 32):
    '''Input Image Size: (227, 227)'''
    super(Model_FER, self).__init__()
    self.conv0 = nn.Conv2d(in_channels = 1, out_channels = 64, kernel_size = 11)
    self.conv1 = nn.Conv2d(in_channels = 64, out_channels = 96, kernel_size = 5)
    self.conv2 = nn.Conv2d(in_channels = 64, out_channels = 128, kernel_size = 5)
    self.conv3 = nn.Conv2d(in_channels = 224, out_channels = 128, kernel_size = 3)
    self.conv4 = nn.Conv2d(128, 64, 3)
    self.batchnorm = nn.BatchNorm2d(64)
    self.dropout1 = nn.Dropout(0.4)
    self.dropout2 = nn.Dropout(0.4)
    self.fc1 = nn.Linear(64, l1)
    self.fc2 = nn.Linear(l1, l2)
    self.fc3 = nn.Linear(l2, 5)
  def forward(self, x, verbose = False):
    x = self.conv0(x)
    x = F.relu(x)
    x = F.max_pool2d(x, kernel_size=3)
    x =  self.batchnorm(x)
    x1 = self.conv1(x)
    x1 = F.relu(x1)
    x1 = F.max_pool2d(x1, kernel_size = 3)
    x2 = self.conv2(x)
    x2 = F.relu(x2)
    x2 = F.max_pool2d(x2, kernel_size = 3)
    x3 = torch.cat((x1, x2), dim = 1)
    x3 = self.dropout1(x3)
    x3 = self.conv3(x3)
    x3 = F.relu(x3)
    x3 = F.max_pool2d(x3, kernel_size = 3)
    x4  = self.conv4(x3)
    x4 = F.relu(x4)
    x = torch.flatten(x4, 1)
    x = self.fc1(x)
    x = F.relu(x)
    x = self.dropout2(x)
    x = self.fc2(x)
    x = F.relu(x)
    x = self.fc3(x)
    x = F.log_softmax(x, dim=1)
    return x

## Test Your Model Here

In [ ]:
alex_train_dataset, alex_val_dataset = get_dataset(greyed_images,labels , (227, 227))
train_dl = DataLoader(alex_train_dataset,batch_size = 5,shuffle = True)
for i in train_dl:
  image, label = i
  break


In [ ]:
model = Model_FER()
model.to(device)
label = label.to(device)
model(image.to(device))
# output = model(image.to(device))

# Training And Validation and Testing on real data

In [ ]:
def train(model, dataset, epochs, criterion, optimizer, batch_size = 50):
  train_dl = DataLoader(dataset,batch_size = batch_size,shuffle = True)
  length = len(train_dl)
  data_length = dataset.__len__()
  history = {}
  history['train_acc'] = []
  history['train_loss'] = []
  history['val_acc'] = []
  history['val_loss'] = []
  for epoch in range(epochs):
    running_loss = 0.0
    print("Epoch: {}".format(epoch+1))
    for i, data in enumerate(train_dl):
      progress_batch(i,length)
      image,label = data[0].to(device),data[1].to(device)
      optimizer.zero_grad()
      outputs = model(image)
      loss = criterion(outputs, label)
      loss.backward()
      optimizer.step()
      running_loss+=loss.item()
    running_loss/=data_length
    print("   Loss:",f' {running_loss:.5f}')
    train_loss, train_acc = evaluate(Alexmodel, alex_train_dataset, Alexcriterion)
    val_loss, val_acc = evaluate(Alexmodel,alex_val_dataset, Alexcriterion)
    history['train_acc'].append(train_acc)
    history['train_loss'].append(train_loss)
    history['val_loss'].append(val_loss)
    history['val_acc'].append(val_acc)
    print(f'\n train_loss: {train_loss:.4f} :: train_acc: {train_acc:.2f} :: val_loss: {val_loss:.4f} :: val_acc: {val_acc:.2f}')
  print("Finished Training")
  return history

In [ ]:
def evaluate(model,dataset,criterion, batch_size = 50):
  correct = 0
  total = dataset.__len__()
  total_loss = 0
  val_dl = DataLoader(dataset,batch_size = batch_size)
  with torch.no_grad():
    for data in val_dl:
      image,label = data[0].to(device),data[1].to(device)
      outputs = model(image)
      loss = criterion(outputs,label)
      outputs = outputs.detach()
      outputs = torch.argmax(outputs, dim = 1)
      result = torch.sum(outputs==label)
      correct+=result.item()
      total_loss+=loss.item()
  total_loss/=total
  accuracy = correct/total
  return total_loss, accuracy

```
Alexmodel = Model_FER()
Alexmodel.to(device)
Alexcriterion = nn.NLLLoss()
Alexoptimizer = optim.Adadelta(Alexmodel.parameters(), lr=1e-1, weight_decay=1e-5)
```

**gives** (30 iterations only)

```
Train Accuracy: 
Accuracy: 0.8811363636363636 || Loss: 0.00647
Accuracy: 0.7254545454545455 || Loss: 0.01646

```

In [ ]:
Alexmodel = Model_FER()
Alexmodel.to(device)
Alexcriterion = nn.NLLLoss()
Alexoptimizer = optim.Adadelta(Alexmodel.parameters(), lr=1e-1, weight_decay=1)
alex_train_dataset, alex_val_dataset = get_dataset(greyed_images, labels, (227, 227))
history = train(Alexmodel, alex_train_dataset,20, Alexcriterion, Alexoptimizer,30)

In [ ]:
plt.title("Accuracies:")
plt.plot(history['train_acc'], label = 'train_acc')
plt.plot(history['val_acc'], label ='val_acc')
plt.legend()
plt.show()
plt.title("Losses:")
plt.plot(history['train_loss'], label = 'train_loss')
plt.plot(history['val_loss'], label = 'val_loss')
plt.legend()
plt.show()

In [ ]:
print("Train Accuracy: ")
evaluate(Alexmodel, alex_train_dataset, Alexcriterion)
print("Validation Accuracy:")
evaluate(Alexmodel, alex_val_dataset, Alexcriterion)

In [ ]:
from datetime import datetime
now = datetime.now()
current_time = now.strftime("%H:%M:%S")
torch.save(Alexmodel, "/content/drive/MyDrive/Beauty/beauty_model"+current_time+".pt")

In [ ]:
#Testing part.
IMG_ADDRESS = "/content/Screenshot from 2021-02-03 22-15-52.png"
face_cascade = cv2.CascadeClassifier("/content/haarcascade_frontalface_default.xml")
eye_cascade = cv2.CascadeClassifier("/content/haarcascade_eye.xml")
img = cv2.imread(IMG_ADDRESS)
img = cv2.resize(img, (480, 480), interpolation = cv2.INTER_AREA)
img = rotate(img)
img = crop(img)
img = cv2.resize(img, (227, 227), interpolation = cv2.INTER_AREA)
img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY);

In [ ]:
img = img.reshape(1,1,227,227)

In [ ]:
img = torch.tensor(img, device = device, dtype= torch.float)
outputs = Alexmodel(img)

In [ ]:
torch.argmax(outputs, dim =1)

In [ ]:
plt.imshow(np.array(img.cpu())[0,0])

In [ ]:
def rotate(image):
  l1=[]
  gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  faces = face_cascade.detectMultiScale(gray)
  if(len(faces)==1):
    for (x,y,w,h) in faces:
      face_detected = gray[y:y+h,x:x+h]
      eyes = eye_cascade.detectMultiScale(face_detected)
      for (ex,ey,ew,eh) in eyes:
          l1.append((ex+ew/2,ey+eh/2))
      
      if(len(l1)==2):
        dist_x = l1[1][0]-l1[0][0]
        dist_y = l1[1][1] - l1[0][1]
        if dist_x<0:
          dist_y = -dist_y
        dist_x = np.abs(dist_x)     
        angle = np.arctan(dist_y/dist_x) * 180/3.14
        M = cv2.getRotationMatrix2D((240, 240), angle, 1.0)
        image = cv2.warpAffine(image, M,(480,480))
  return image

def crop(image,x_factor=2.5,y_factor=3.5):
  l1=[]
  gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  faces = face_cascade.detectMultiScale(gray)
  if(len(faces)==1):
    for (x,y,w,h) in faces:
      gray = gray[y:y+h,x:x+h]
      image = image[y:y+h, x:x+h]
  eyes = eye_cascade.detectMultiScale(gray)
  for (ex,ey,ew,eh) in eyes:
      l1.append((ex+ew/2,ey+eh/2))
  if(len(l1)==2):
    dist = np.sqrt((l1[0][0]-l1[1][0])**2+(l1[0][1]-l1[1][1])**2)
    center_x = image.shape[1]//2
    center_y = image.shape[0]//2
    shift_x = int(dist*x_factor)//2
    shift_y = int(dist*y_factor)//2
    start_x = center_x - shift_x
    start_x = max(start_x,0)
    end_x = center_x+shift_x
    end_x = min(end_x,image.shape[1])
    start_y = center_y - shift_y
    start_y = max(start_y,0)
    end_y = center_y + shift_y
    end_y = min(end_y,image.shape[0])
    image = image[start_y:end_y,start_x:end_x]
  return image

In [ ]:
IMG_ADDRESS = "/content/Screenshot from 2021-02-03 22-15-52.png"

In [ ]:
img = cv2.imread(IMG_ADDRESS)
plt.imshow(img)

In [ ]:
def rotate(image):
  l1=[]
  gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  faces = face_cascade.detectMultiScale(gray)
  if(len(faces)==1):
    for (x,y,w,h) in faces:
      face_detected = gray[y:y+h,x:x+h]
      eyes = eye_cascade.detectMultiScale(face_detected)
      print(eyes)
      for (ex,ey,ew,eh) in eyes:
          l1.append((ex+ew/2,ey+eh/2))
      
      if(len(l1)==2):
        dist_x = l1[1][0]-l1[0][0]
        dist_y = l1[1][1] - l1[0][1]
        if dist_x<0:
          dist_y = -dist_y
        dist_x = np.abs(dist_x)     
        angle = np.arctan(dist_y/dist_x) * 180/3.14
        M = cv2.getRotationMatrix2D((240, 240), angle, 1.0)
        image = cv2.warpAffine(image, M,(480,480))
  return image
image = rotate(img)
plt.imshow(image)

In [ ]:
def crop(image,x_factor=2,y_factor=3):
  l1=[]
  gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  faces = face_cascade.detectMultiScale(gray)
  if(len(faces)==1):
    for (x,y,w,h) in faces:
      gray = gray[y:y+h,x:x+h]
      image = image[y:y+h, x:x+h]
  eyes = eye_cascade.detectMultiScale(gray)
  for (ex,ey,ew,eh) in eyes:
      l1.append((ex+ew/2,ey+eh/2))
  if(len(l1)==2):
    dist = np.sqrt((l1[0][0]-l1[1][0])**2+(l1[0][1]-l1[1][1])**2)
    center_x = image.shape[1]//2
    center_y = image.shape[0]//2
    shift_x = int(dist*x_factor)//2
    shift_y = int(dist*y_factor)//2
    start_x = center_x - shift_x
    start_x = max(start_x,0)
    end_x = center_x+shift_x
    end_x = min(end_x,image.shape[1])
    start_y = center_y - shift_y
    start_y = max(start_y,0)
    end_y = center_y + shift_y
    end_y = min(end_y,image.shape[0])
    image = image[start_y:end_y,start_x:end_x]
  return image
img_crop = crop(image)
plt.imshow(img_crop)

# Hyperparameter optimisation

In [ ]:
def train_optim(config, checkpoint_dir=None):
  net = Model_FER(config['l1'], config['l2'])
  net.to(device)
  criterion = nn.NLLLoss()
  optimizer = optim.Adadelta(net.parameters(), lr=config['lr'], weight_decay=1e-5)
  if checkpoint_dir:
    model_state, optimizer_state = torch.load(
        os.path.join(checkpoint_dir, "checkpoint"))
    net.load_state_dict(model_state)
    optimizer.load_state_dict(optimizer_state)
  train_dataset, val_dataset = get_dataset(greyed_images, labels, (227, 227))
  trainloader = DataLoader(train_dataset,batch_size = int(config['batch_size']),shuffle = True, num_workers=8)
  valloader = DataLoader(val_dataset,batch_size = int(config['batch_size']),shuffle = True, num_workers=8)
  for epoch in range(10):  # loop over the dataset multiple times
    running_loss = 0.0
    epoch_steps = 0
    for i, data in enumerate(trainloader, 0):
      # get the inputs; data is a list of [inputs, labels]
      inputs, labels = data
      inputs, labels = inputs.to(device), labels.to(device)

      # zero the parameter gradients
      optimizer.zero_grad()

      # forward + backward + optimize
      outputs = net(inputs)
      loss = criterion(outputs, labels)
      loss.backward()
      optimizer.step()

      # print statistics
      running_loss += loss.item()
      epoch_steps += 1
      if i % 2000 == 1999:  # print every 2000 mini-batches
          print("[%d, %5d] loss: %.3f" % (epoch + 1, i + 1,
                                          running_loss / epoch_steps))
          running_loss = 0.0

    # Validation loss
    val_loss = 0.0
    val_steps = 0
    total = 0
    correct = 0
    for i, data in enumerate(valloader, 0):
      with torch.no_grad():
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        outputs = net(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        loss = criterion(outputs, labels)
        val_loss += loss.cpu().numpy()
        val_steps += 1

    with tune.checkpoint_dir(epoch) as checkpoint_dir:
      path = os.path.join(checkpoint_dir, "checkpoint")
      torch.save((net.state_dict(), optimizer.state_dict()), path)

    tune.report(loss=(val_loss / val_steps), accuracy=correct / total)
  print("Finished Training")

In [ ]:
def test_accuracy(net, device="cpu"):
  train_dataset, val_dataset = get_dataset(greyed_images, labels, (227, 227))
  valloader = DataLoader(val_dataset,batch_size = 32,shuffle = True, num_workers=8)
  correct = 0
  total = 0
  with torch.no_grad():
    for data in testloader:
      images, labels = data
      images, labels = images.to(device), labels.to(device)
      outputs = net(images)
      _, predicted = torch.max(outputs.data, 1)
      total += labels.size(0)
      outputs = outputs.detach()
      outputs = torch.argmax(outputs, dim = 1)
      result = torch.sum(outputs==label)
      correct+=result.item()
  return correct / total

In [ ]:
def main_optimise(num_samples=10, max_num_epochs=10, gpus_per_trial=1):
    config = {
        "l1": tune.sample_from(lambda _: 2 ** np.random.randint(2, 9)),
        "l2": tune.sample_from(lambda _: 2 ** np.random.randint(2, 9)),
        "lr": tune.loguniform(1e-4, 1e-1),
        "batch_size": tune.choice([8, 16, 32, 64])
    }
    scheduler = ASHAScheduler(
        metric="loss",
        mode="min",
        max_t=max_num_epochs,
        grace_period=1,
        reduction_factor=2)
    reporter = CLIReporter(
        # parameter_columns=["l1", "l2", "lr", "batch_size"],
        metric_columns=["loss", "accuracy", "training_iteration"])
    result = tune.run(
        partial(train_optim),
        resources_per_trial={"cpu": 2, "gpu": gpus_per_trial},
        config=config,
        num_samples=num_samples,
        scheduler=scheduler,
        progress_reporter=reporter)

    best_trial = result.get_best_trial("loss", "min", "last")
    print("Best trial config: {}".format(best_trial.config))
    print("Best trial final validation loss: {}".format(
        best_trial.last_result["loss"]))
    print("Best trial final validation accuracy: {}".format(
        best_trial.last_result["accuracy"]))

    best_trained_model = Net(best_trial.config["l1"], best_trial.config["l2"])
    best_trained_model.to(device)

    best_checkpoint_dir = best_trial.checkpoint.value
    model_state, optimizer_state = torch.load(os.path.join(
        best_checkpoint_dir, "checkpoint"))
    best_trained_model.load_state_dict(model_state)

    test_acc = test_accuracy(best_trained_model, device)
    print("Best trial test set accuracy: {}".format(test_acc))

In [ ]:
if __name__ == "__main__":
  main_optimise()